upload packages

In [1]:
library(readr)
library(org.Hs.eg.db)
library("vsn")
library(DESeq2)

Loading required package: AnnotationDbi
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: Biobase
Welcome

load the mRNA-Seq data 

In [2]:
data.path="RNA"
files <- list.files(path=data.path,recursive=T, pattern = "gz")

read the first file for the first time

In [3]:
file=files[1]
file.id=strsplit(file,"/")[[1]][1]

open a connection to your gz file and read the file

In [4]:
gz.con=gzfile(file.path(data.path,files[1]))
temp <- read.table(gz.con, header=F)

create a storing object mrna.exp to save the whole read counts of each file read in an iteration

In [5]:
mrna.exp=temp
rownames(mrna.exp)=mrna.exp[,1]
mrna.exp=mrna.exp[-1]
colnames(mrna.exp)=c(file.id)

loop for reading other files

In [6]:
for(i in 2: length(files))
  {
    
    ## refer to the next file (note that we start from index 2, bec we already read the first file)
    file=files[i]
    file.id=strsplit(file,"/")[[1]][1]
  
    # read the next file  
    gz.con=gzfile(file.path(data.path,files[i]))
    temp <- read.table(gz.con, header=F)
    
    ## remove the first column, bec we had it already
    temp=temp[-1]
    colnames(temp)=c(file.id)
    
    mrna.exp=cbind(mrna.exp,temp)
  }
print(mrna.exp)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



do the mapping of ensembel.id to gene symbol and their adjustment


In [7]:
#prepare the ensembel id to be like the one in the database
ensemble.id=sapply(rownames(mrna.exp), function(x) strsplit(as.character(x),"\\.")[[1]][1])
View(ensemble.id)
mrna.exp=cbind(ensemble.id,mrna.exp)

mapper<- mapIds(org.Hs.eg.db, keys=ensemble.id, column="SYMBOL",keytype="ENSEMBL", multiVals="first") #??????
mapper.df=as.data.frame(mapper)

mapper.df=cbind(rownames(mapper.df), mapper.df)
names(mapper.df)=c("ensemble.id","symbol")
                   
# prepare the ensembel id to be like the one in the database
ensemble.id=sapply(rownames(mrna.exp), function(x) strsplit(as.character(x),"\\.")[[1]][1])
View(ensemble.id)
mrna.exp=cbind(ensemble.id,mrna.exp)

mapper<- mapIds(org.Hs.eg.db, keys=ensemble.id, column="SYMBOL",keytype="ENSEMBL", multiVals="first") #??????
mapper.df=as.data.frame(mapper)

mapper.df=cbind(rownames(mapper.df), mapper.df)
names(mapper.df)=c("ensemble.id","symbol")


mrna.exp2=merge(mrna.exp,mapper.df,by="ensemble.id",all.x=T) #????? ensembl.id of y
# drop the first column (ensemble.id)
mrna.exp2=mrna.exp2[-1]

mrna.exp2=mrna.exp2[ ! is.na(mrna.exp2$symbol),]

# check duplciation of of gene symbols?  
x=duplicated(mrna.exp2$symbol)  
sum(x)

### yes .. why ? transcripts?  solutions : aggregation
mrna.exp.data=mrna.exp2[-dim(mrna.exp2)[2]]
mrna.exp.data=apply(mrna.exp.data,2, as.numeric)

####remove  duplication by aggregation
mrna.exp.data.agg= aggregate(mrna.exp.data, list(mrna.exp2$symbol),FUN=mean)

rownames(mrna.exp.data.agg)=mrna.exp.data.agg$Group.1
mrna.exp.data.agg=mrna.exp.data.agg[-1]

file.ids=colnames(mrna.exp.data.agg)


ERROR: Error in View(ensemble.id): ‘View()’ not yet supported in the Jupyter R kernel


load the mrna sample sheets

In [9]:
pheno <- read_delim("RNA.tsv", "\t", escape_double = FALSE, trim_ws = TRUE)
print(pheno)

Parsed with column specification:
cols(
  File.ID = col_character(),
  File.Name = col_character(),
  Data.Category = col_character(),
  Data.Type = col_character(),
  Project.ID = col_character(),
  Case.ID = col_character(),
  Sample.ID = col_character(),
  Sample.Type = col_character()
)


# A tibble: 551 x 8
   File.ID File.Name Data.Category Data.Type Project.ID Case.ID Sample.ID
   <chr>   <chr>     <chr>         <chr>     <chr>      <chr>   <chr>    
 1 cbb56e… 5a7f277a… Transcriptom… Gene Exp… TCGA-LUSC  TCGA-6… TCGA-66-…
 2 d0d1b0… 94b65d9b… Transcriptom… Gene Exp… TCGA-LUSC  TCGA-2… TCGA-22-…
 3 98e317… 9ef0c6dd… Transcriptom… Gene Exp… TCGA-LUSC  TCGA-8… TCGA-85-…
 4 574abc… 00068002… Transcriptom… Gene Exp… TCGA-LUSC  TCGA-5… TCGA-56-…
 5 0e18c3… 01444ada… Transcriptom… Gene Exp… TCGA-LUSC  TCGA-7… TCGA-77-…
 6 58c0f4… 88cf0020… Transcriptom… Gene Exp… TCGA-LUSC  TCGA-7… TCGA-77-…
 7 4b6076… 9813b5d6… Transcriptom… Gene Exp… TCGA-LUSC  TCGA-6… TCGA-60-…
 8 92ab7a… 71550bf2… Transcriptom… Gene Exp… TCGA-LUSC  TCGA-3… TCGA-33-…
 9 220a03… 60040f95… Transcriptom… Gene Exp… TCGA-LUSC  TCGA-1… TCGA-18-…
10 5c3dd6… 60a2aca4… Transcriptom… Gene Exp… TCGA-LUSC  TCGA-4… TCGA-43-…
# … with 541 more rows, and 1 more variable: Sample.Type <chr>


#we will rename the columns of our exp data with the sample ids columns of the pheno file
however we need to match the file ids to 

In [10]:
file.ids.pheno=pheno$File.ID
index.files=match(file.ids,file.ids.pheno)
names(mrna.exp.data.agg)=pheno$Sample.ID[index.files]

ERROR: Error in match(file.ids, file.ids.pheno): object 'file.ids' not found
